**Preparing Environment**

In [1]:
import os
import time
from PIL import Image, ImageFilter
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import random
import cv2


In [3]:
os.environ["TFHUB_DOWNLOAD_PROGRESS"] = "True"
# Declaring Constants
IMAGE_PATH = "D:/MachineLearningData/MNSTP/mnist_p/testing/0/1.png"
SAVED_MODEL_PATH = "https://tfhub.dev/captain-pool/esrgan-tf2/1"

In [6]:
img = Image.open('D:/MachineLearningData/MNSTP/mnist_p/normal/training/0/1.png')
img=img.resize((128,128),Image.BICUBIC)
img.save("%s" % "0control.png")
img.show()


In [11]:
 def add_noise(img,mean,sigma,size):
        x,y=size
        img=img/255
        n=np.random.normal(loc=mean,scale=sigma,size=(x,y))
        return (img+n)*255
    
def preprocess_image(image_path):
    hr_image = tf.image.decode_image(tf.io.read_file(image_path))
  # If PNG, remove the alpha channel. The model only supports
  # images with 3 color channels.abs
    if hr_image.shape[-1] == 4:
        hr_image = hr_image[...,:-1]
    hr_size = (tf.convert_to_tensor(hr_image.shape[:-1]) // 4) * 4
    hr_image = tf.image.crop_to_bounding_box(hr_image, 0, 0, hr_size[0], hr_size[1])
    hr_image = tf.cast(hr_image, tf.float32)
    return tf.expand_dims(hr_image, 0)

def save_image(image, filename):
  if not isinstance(image, Image.Image):
    image = tf.clip_by_value(image, 0, 255)
    image = Image.fromarray(tf.cast(image, tf.uint8).numpy())
  image.save("%s" % filename)
  print("Saved as %s" % filename)



In [2]:
def downscaleDataset(path,destination):
           
            for filename in os.listdir(path):
                print(filename+"/10")
                f=os.path.join(path,filename)#full path
                for file in os.listdir(f):
                    img = Image.open(os.path.join(f,file))
                    img=img.resize((56,56),Image.BICUBIC)
                    img.save("%s" % destination+filename+"/"+file)
            print("Downscaling complete")
    

In [3]:
def addNoiseToDataSet(path,destination):
    print("starting")
    for filename in os.listdir(path):
            print(filename+"/10")
            f=os.path.join(path,filename)#full path
            for file in os.listdir(f):
                image=cv2.imread( os.path.join(f,file),0)
                image=add_noise(image,sigma=0.15,mean=0)
                cv2.imwrite(destination+filename+"/"+file,image)
    print("Noising complete")

In [4]:
def upscaleDataset(path,destination):
    print("loading model")
    model = hub.load(SAVED_MODEL_PATH)
    print("starting")
    for filename in os.listdir(path):
        if(int(filename)==4):
            print(filename+"/10")
            f=os.path.join(path,filename)#full path
            for file in os.listdir(f):
                hr_image=preprocess_image( os.path.join(f,file))
                image3channel = tf.image.grayscale_to_rgb(hr_image)
                fake_image = model(image3channel)
                fake_image = tf.squeeze(fake_image)
                save_image(fake_image, filename=destination+filename+"/"+file)
        print("upscaling complete")

In [ ]:
upscaleDataset("D:/MachineLearningData/MNSTP/mnist_p/testing/",
               "D:/MachineLearningData/MNSTP/mnist_p//testingUpscaled/")

In [ ]:
 addNoiseToDataSet("D:/MachineLearningData/MNSTP/mnist_p/Upscaled/train/",
               "D:/MachineLearningData/MNSTP/mnist_p/Upscaled/trainNoise/")

In [7]:
downscaleDataset("D:/MachineLearningData/MNSTP/mnist_p/Normal/train/",
               "D:/MachineLearningData/MNSTP/mnist_p/Control/trainUpscaledFake/")

0/10
1/10
2/10
3/10
4/10
5/10
6/10
7/10
8/10
9/10
Downscaling complete
